In [1]:
import os
import sys
import pickle
import pandas as pd
# autoreload
%load_ext autoreload
%autoreload 2

In [2]:
# Obtener la ruta del directorio base (nivel superior del proyecto)
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
SRC_PATH = os.path.join(BASE_DIR, "src")

# Agregar SRC_PATH a sys.path si no está presente
if SRC_PATH not in sys.path:
    sys.path.insert(0, SRC_PATH)

In [3]:
with open(r'..\models\RandomForestClassifier.pkl', 'rb') as archivo:
    modelo = pickle.load(archivo)


In [4]:
# Importar librerías necesarias
# Cargar datasets
df_tr = pd.read_csv(r"..\data\raw\prueba_op_base_pivot_var_rpta_alt_enmascarado_trtest.csv")[[
    'nit_enmascarado', 'num_oblig_orig_enmascarado', 'num_oblig_enmascarado', 'var_rpta_alt'
]]
df_hist_scores = pd.read_csv(r"..\data\raw\prueba_op_probabilidad_oblig_base_hist_enmascarado_completa.csv")
df_clientes = pd.read_csv(r"..\data\raw\prueba_op_master_customer_data_enmascarado_completa.csv")
df_pagos = pd.read_csv(r"..\data\raw\prueba_op_maestra_cuotas_pagos_mes_hist_enmascarado_completa.csv")

# Ajustar formato de 'fecha_corte' en df_pagos y df_clientes
df_pagos['fecha_corte'] = df_pagos['fecha_corte'].astype(str).str[:6].astype(int)
df_clientes['fecha_corte'] = df_clientes['year'].astype(str) + df_clientes['month'].astype(str).str.zfill(2)

# Ordenar y eliminar duplicados en df_hist_scores
df_hist_scores.sort_values(by=['fecha_corte'], ascending=False, inplace=True)
df_hist_scores.drop_duplicates(subset=['nit_enmascarado'], keep='first', inplace=True)

# Ordenar y eliminar duplicados en df_clientes
df_clientes.sort_values(by=['fecha_corte'], ascending=False, inplace=True)
df_clientes.drop_duplicates(subset=['nit_enmascarado'], keep='first', inplace=True)

# Ordenar y eliminar duplicados en df_pagos
df_pagos.sort_values(by=['fecha_corte'], ascending=False, inplace=True)
df_pagos.drop_duplicates(subset=['nit_enmascarado'], keep='first', inplace=True)

df_oot = pd.read_csv(r'..\data\raw\prueba_op_base_pivot_var_rpta_alt_enmascarado_oot.csv')

df_test = pd.merge(
    df_oot, df_clientes, 
    on=['nit_enmascarado'], 
    how='left'
)
df_test = pd.merge(
    df_test, df_pagos, 
    on=['nit_enmascarado'], 
    how='left'
)

df_test = pd.merge(
    df_test, df_hist_scores, 
    on=['nit_enmascarado'], 
    how='left'
)

df_test['id'] = df_test['nit_enmascarado'].astype(str) + '#' + df_test['num_oblig_orig_enmascarado'].astype(str) + '#' + df_test['num_oblig_enmascarado'].astype(str)
df_test

,nit_enmascarado,num_oblig_orig_enmascarado,num_oblig_enmascarado_x,fecha_var_rpta_alt,cod_tipo_doc,tipo_cli,ctrl_terc,genero_cli,ano_nac_cli,edad_cli,...,porc_pago,marca_pago,ajustes_banco,num_oblig_enmascarado,fecha_corte,lote,prob_propension,prob_alrt_temprana,prob_auto_cura,id
0,257335,444821,635511,202401,1.0,PERSONA NATURAL,CLIENTE,F,1973.0,50.0,...,0.0,NO_PAGO,NO,635511.0,202312.0,1.0,0.733536,0.424859,0.697358,257335#444821#635511.0
1,59584,350400,730364,202401,1.0,PERSONA NATURAL,CLIENTE,M,1965.0,58.0,...,8.0,PAGO_MENOS,NO,730364.0,202312.0,2.0,0.741083,0.203964,0.787728,59584#350400#730364.0
2,397604,973821,106521,202401,1.0,PERSONA NATURAL,CLIENTE,F,1988.0,35.0,...,5.0,PAGO_MENOS,NO,106513.0,202312.0,1.0,0.217455,0.715489,0.199324,397604#973821#106513.0
3,368086,382995,696856,202401,1.0,PERSONA NATURAL,CLIENTE,M,1998.0,25.0,...,0.0,NO_PAGO,NO,696856.0,202312.0,1.0,0.742705,0.491246,0.372921,368086#382995#696856.0
4,255009,434238,645924,202401,NaN,NaN,NaN,NaN,NaN,NaN,...,506.0,PAGO_MENOS,NO,953589.0,202312.0,3.0,0.289970,0.597897,0.299553,255009#434238#953589.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112544,316499,14503,1074624,202401,3.0,PERSONA JURíDICA,CLIENTE,NaN,NaN,NaN,...,111.0,PAGO_MENOS,NO,536986.0,202312.0,1.0,0.650076,0.825191,0.721970,316499#14503#536986.0
112545,254311,30479,1013957,202401,1.0,PERSONA NATURAL,CLIENTE,M,1966.0,57.0,...,200.0,FACTURACION_MES_SGTE,NO,486272.0,202312.0,3.0,0.736331,0.581485,0.476163,254311#30479#486272.0
112546,483193,13732,1006779,202401,1.0,PERSONA NATURAL,CLIENTE,M,1988.0,34.0,...,0.0,NO_PAGO,NO,310671.0,202312.0,1.0,0.244340,0.647481,0.216163,483193#13732#310671.0
112547,293824,22004,1062743,202401,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NO_PAGO,NO,1062743.0,202312.0,3.0,0.117078,0.552449,0.581126,293824#22004#1062743.0


In [6]:
df_test.shape

(112549, 60)

In [5]:
df_oot.shape

(112549, 4)

In [7]:
features = pd.read_csv(r'..\data\procesed\features.csv')

In [8]:
from data_engineer import FeatureSelector

limpieza = FeatureSelector(df_test, features)
df = limpieza.fit_transform()

Limpiando variables numéricas...


d:\Desktop\juan_guzman_prueba\env\lib\site-packages\pandas\core\nanops.py:1256: RuntimeWarning: invalid value encountered in subtract
  adjusted = values - mean


Modo inferencia: No se encontró 'var_rpta_alt'
Detectando tipos de variables...
Preprocesando variables...
Transformando variables...
Reconstruyendo nombres de columnas...
Seleccionando variables...
Ajustando columnas faltantes...
Ordenando columnas...
Finalizando DataFrame...


In [9]:
# 2. Seleccionar columnas que usó el modelo
columnas_modelo = modelo.feature_names_in_
X = df[columnas_modelo]

# 3. Predecir
predicciones = modelo.predict(X)

# 4. Crear DataFrame final
df_submission = pd.DataFrame({
    'id': df['id'],
    'var_rpta_alt': predicciones,
    'var_rpta_alt_prob': modelo.predict_proba(X)[:, 1]
})

# 5. Guardar en CSV con el formato solicitado
df_submission.to_csv('submission.csv', index=False)


In [10]:
df_submission

,id,var_rpta_alt,var_rpta_alt_prob
0,257335#444821#635511.0,0.0,0.32
1,59584#350400#730364.0,0.0,0.38
2,397604#973821#106513.0,0.0,0.34
3,368086#382995#696856.0,1.0,0.58
4,255009#434238#953589.0,0.0,0.33
...,...,...,...
112544,316499#14503#536986.0,0.0,0.20
112545,254311#30479#486272.0,0.0,0.41
112546,483193#13732#310671.0,0.0,0.31
112547,293824#22004#1062743.0,0.0,0.30
